# ANÁLISES EM AC2

Análsies realizadas sob rede obtida por meio da extração no WoS (BIOLOGIA MATEMÁTICA E COMPUTACIONAL).

In [1]:
# carregando bibliotecas

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
pal = 'GnBu_d'
%matplotlib inline

import networkx as nx
from collections import Counter

import gc
import os

import plotly.plotly as py
import plotly.graph_objs as go
import plotly.offline as offline
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
#init_notebook_mode(connected=True)
offline.init_notebook_mode()

from IPython.html.widgets import * # interact

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.
C:\Users\Walter\Anaconda3\lib\site-packages\IPython\html.py:14: ShimWarning:

The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.



In [2]:
# importando os dados

citations = pd.read_csv('outputs/citations.csv', index_col=0, encoding = "ISO-8859-1")
journals_cit = pd.read_csv('outputs/journals_cit.csv', index_col=0, encoding = "ISO-8859-1")
articles_cit = pd.read_csv('outputs/articles_cit.csv', index_col=0, encoding = "ISO-8859-1")

In [3]:
citations.head()

,SOURCE_A,SOURCE_J,TARGET_A,TARGET_J,YEAR
1,WOS:000418951400033,1553-734X,WOS:A1997XU79300002,0305-1048,2017
2,WOS:000418951400033,1553-734X,WOS:000382258600111,1932-6203,2017
3,WOS:000418951400033,1553-734X,WOS:000354024900017,0969-2126,2017
4,WOS:000418951400033,1553-734X,WOS:000176271000003,0907-4449,2017
5,WOS:000418951400033,1553-734X,WOS:000084896300069,0305-1048,2017


In [4]:
journals_cit.head()

,TITLE,CITED_OUT,CITED_IN,TOT_ART
ISSN,,,,
1553-734X,PLoS Computational Biology,224085.0,12190.0,4088.0
1367-4803,BIOINFORMATICS,132071.0,52320.0,5840.0
1752-0509,BMC Systems Biology,65446.0,4328.0,1443.0
1471-2105,BMC BIOINFORMATICS,149982.0,22495.0,4742.0
0303-6812,JOURNAL OF MATHEMATICAL BIOLOGY,27093.0,6551.0,845.0


In [240]:
articles_cit.head()

,ID_WOS,TITLE,JOURNAL_ID,YEAR,CITED_IN,CITED_OUT,CITED_OUT_OB,CITED_OUT_T
1,WOS:000418951400033,High-resolution global peptide-protein docking...,1,2017,0,62,3,62
2,WOS:000418951400005,A cyber-linked undergraduate research experien...,1,2017,0,34,7,47
3,WOS:000418951400031,Physiological models of the lateral superior o...,1,2017,0,170,3,185
4,WOS:000418951400028,MAGPIE: Simplifying access and execution of co...,1,2017,0,24,3,28
5,WOS:000418951400009,Stabilizing patterns in time: Neural network a...,1,2017,0,26,5,30


## 1. Identificando os periodicos periféricos

* Na rede AC2, identificar os periódicos periféricos que irão seguir o comportamento estratégico.


* Na tese do Walter, os periódicos periféricos foram defidos como os que possuem o seu respectivo FI2 (fator de impacto - 2 anos) no último quartil. 

Logo, a ideia aqui é calcular o FI2 de 2017 para cada periódico. Em seguida ordenar esses valores e encontra o último quartil. 

### Calculando o FI2

O WoS possui uma tabela com FI2 calculado. No entanto, vamos recalcular esses valores para a base extraída, pois ela é apenas uma fatia da rede utilizada para o cálculo.

In [241]:
# FI calculado pelo WoS

FI = pd.read_csv('outputs/IF.csv', index_col=0)
FI.head()

,TITLE,2010,2011,2012,2013,2014,2015,2016
1,PLoS Computational Biology,5.515,5.215,4.867,4.829,4.620,4.587,4.542
2,BIOINFORMATICS,4.877,5.468,5.323,4.621,4.981,5.766,7.307
3,BMC Systems Biology,3.565,3.148,2.982,2.853,2.435,2.208,2.303
4,BMC BIOINFORMATICS,3.029,2.751,3.024,2.672,2.576,2.435,2.448
5,JOURNAL OF MATHEMATICAL BIOLOGY,3.021,2.963,2.366,2.388,1.846,1.716,1.566


In [149]:
FI2 = pd.DataFrame(columns = ['TITLE', 'ISSN', '2017'])
FI2['TITLE'] = journals_title
FI2['ISSN'] = list(journals_cit.index[0:59])
FI2.head()

,TITLE,ISSN,2017
0,PLoS Computational Biology,1553-734X,NaN
1,BIOINFORMATICS,1367-4803,NaN
2,BMC Systems Biology,1752-0509,NaN
3,BMC BIOINFORMATICS,1471-2105,NaN
4,JOURNAL OF MATHEMATICAL BIOLOGY,0303-6812,NaN


In [166]:
# funcao para calcular o FI2

def calc_if2(citations, FI2, year):
    
    issn = list(FI2['ISSN'])
    
    for i in range(0,len(FI2)):
        
        # seleciona o periodico
        issn_i = issn[i] 
        
        # conta as citacoes recebidas por artigos publicados nos anos anteriores
        all_cit_in = citations[citations['TARGET_J'] == issn_i]  
        all_cit_in_ant1 = all_cit_in[all_cit_in['YEAR'] == year-1]
        n_cit_in_ant1 = len(all_cit_in_ant1)
        all_cit_in_ant2 = all_cit_in[all_cit_in['YEAR'] == year-2] 
        n_cit_in_ant2 = len(all_cit_in_ant2)
        
        numerador = (n_cit_in_ant1 + n_cit_in_ant2)
        
        # conta os artigos publicados nesses mesmos anos anteriores 
        all_cit_out = citations[citations['SOURCE_J'] == issn_i] 
        all_cit_out_ant1 = all_cit_out[all_cit_out['YEAR'] == year-1]
        n_cit_out_ant1 = len(all_cit_out_ant1)
        all_cit_out_ant2 = all_cit_out[all_cit_out['YEAR'] == year-2] 
        n_cit_out_ant2 = len(all_cit_out_ant2)
        
        denominador = (n_cit_out_ant1 + n_cit_out_ant2)
        
        if denominador == 0:
            FI2['2017'][i] = 'NaN'
        else:
            FI2['2017'][i] = (numerador/denominador)
    

In [167]:
# calculando o FI2 de 2017
calc_if2(citations, FI2, 2017)

Durante o intervalo de tempo extraído, alguns periódicos surgiram e outros "morreram". Portanto, ao calcularmos o FI2 de 2017 para os periódicos extintos, o denominador será zero, pois não produziram nada no biênio anterior. 

* Aos periódicos que não conseguimos calcular o FI2 foi atribuído `NaN`. 

* Apenas não foi possível calcular o FI2 de um periódico! Ou seja, passamos a contar 58 periódicos para selecionar o grupo de periódicos periféricos. 

In [206]:
FI2 = FI2[FI2['2017'] != 'NaN']
FI2.to_csv('outputs/FI2.csv')

### Periódicos Periféricos

In [207]:
FI2_sort = FI2.sort_values(by='2017', ascending=True)
FI2_sort

,TITLE,ISSN,2017
45,Hokkaido Mathematical Journal,0385-4035,0.00397614
57,Computational Intelligence and Neuroscience,1687-5265,0.00765795
47,Journal of Medical Imaging and Health Informatics,2156-7018,0.00982867
51,Interdisciplinary Sciences-Computational Life ...,1913-2751,0.0117599
7,Evolutionary Bioinformatics,1176-9343,0.0123592
29,Computational and Mathematical Methods in Medi...,1748-670X,0.0150555
28,Current Bioinformatics,1574-8936,0.0159934
52,IEEE Journal of Biomedical and Health Informatics,2168-2194,0.0236537
50,Journal of Bioinformatics and Computational Bi...,0219-7200,0.0262876
41,International Journal of Biostatistics,2194-573X,0.0299252


In [204]:
perifericos = FI2_sort[FI2_sort['2017'] <= list(FI2_sort['2017'].quantile([.25]))[0]]
print('Total de Periódicos Periféricos: ' + str(len(perifericos)))
perifericos

Total de Periódicos Periféricos: 15


,TITLE,ISSN,2017
45,Hokkaido Mathematical Journal,0385-4035,0.00397614
57,Computational Intelligence and Neuroscience,1687-5265,0.00765795
47,Journal of Medical Imaging and Health Informatics,2156-7018,0.00982867
51,Interdisciplinary Sciences-Computational Life ...,1913-2751,0.0117599
7,Evolutionary Bioinformatics,1176-9343,0.0123592
29,Computational and Mathematical Methods in Medi...,1748-670X,0.0150555
28,Current Bioinformatics,1574-8936,0.0159934
52,IEEE Journal of Biomedical and Health Informatics,2168-2194,0.0236537
50,Journal of Bioinformatics and Computational Bi...,0219-7200,0.0262876
41,International Journal of Biostatistics,2194-573X,0.0299252


In [244]:
perifericos.to_csv('outputs/perifericos_2017.csv')

## 2. Mudar a base (5 cols para 6 cols)

A ideia é tentar aproveitar os códigos em Matlab desenvolvidos na tese do Walter e no meu TCC. Portanto, há a necessidade dessa mudança na estrutura da base. 

A base gerada (salvo no  `Matlab` em `var_cit`) pelo Walter é formada pelas 6 colunas:
    1. CÓDIGO ARTIGO SOURCE (código sequencial numéico)
    2. ANO ARTIGO SOURCE 
    3. PERIÓDICO DO ARTIGO SOURCE (código sequencial numérico)
    4. CÓDIGO DO ARTIGO TARGET (código sequencial numérico)
    5. ANO ARTIGO TARGET
    6. PERIÓDICO DO ARTIGO TARGET  (código sequencial numérico)
    
 No entanto, a base gerada pela extração do WoS é formada por 5 colunas:
     1. SOURCE_A (ID_WOS do artigo que está citando) - código não sequencial alfanumérico.
     2. SOURCE_J (ISSN do periódico do artigo em questão) - código não sequencial alfanumérico.
     3. TARGET_A (ID_WOS do artigo que está citando) - código não sequencial alfanumérico.
     4. TARGET_J (ISSN do periódico do artigo em questão) - código não sequencial alfanumérico.
     5. YEAR (ano em que ocorreu a citação - ou seja, o ano do artigo SOURCE_A)

### CÓDIGO ARTIGO SOURCE / CÓDIGO ARTIGO TARGET
* Em `articles_cit`, criar um dicionário atribuindo cada codigo alfanumérico ID_WOS a um código numérico sequencial. 

### PERIÓDICO DO ARTIGO SOURCE / PERIÓDICO DO ARTIGO TARGET
* Em `journals_cit`, criar um dicionário atribuindo cada codigo alfanumérico ISSN a um código numérico sequencial.

### ANO ARTIGO TARGET
* Em `articles_cit`, extrair o ano do artigo `TARGET_A` por meio da busca do `ID_WOS`. 

In [234]:
len(articles_cit)

3956082

In [235]:
len(journals_cit)

61107

In [238]:
cols = articles_cit.columns
[len(articles_cit[col].unique()) for col in cols]

[690445, 689677, 60, 74, 194, 205, 46, 228]